In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import tetrion.plots as plots
import tetrion.db
import tetrion.commands as cmd
import tetrion.options
import tetrion.clientcore
import seaborn as sns
from collections import defaultdict
import numpy as np
import warnings
warnings.simplefilter("ignore")
%matplotlib inline 
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
import redis
import json

In [2]:
df_book_TMF = cmd.book_printer_v2('FUT_TAIFEX_TMF:202504', dt.date(2025,3,4),source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ',depth=5)
df_book_TMF['value']=1
df_book_TMF['sign']=df_book_TMF['value'].cumsum()
df_tick_TMF = cmd.tick_printer('FUT_TAIFEX_TMF:202504',dt.date(2025,3,4),source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ')


In [3]:
pd.set_option('display.max_rows',None)

In [4]:
df_book_TMF.columns

Index(['bid_sz4', 'bid_sz3', 'bid_sz2', 'bid_sz1', 'bid_sz', 'bid_px4',
       'bid_px3', 'bid_px2', 'bid_px1', 'bid_px', 'mid_px', 'ask_px',
       'ask_px1', 'ask_px2', 'ask_px3', 'ask_px4', 'ask_sz', 'ask_sz1',
       'ask_sz2', 'ask_sz3', 'ask_sz4', 'value', 'sign'],
      dtype='object')

In [56]:
# 假设 df 已经只剩下这些列
price_cols = ['bid_px4', 'bid_px3', 'bid_px2', 'bid_px1', 'bid_px']
size_cols  = ['bid_sz4', 'bid_sz3', 'bid_sz2', 'bid_sz1', 'bid_sz']

# 1) 计算分子：各档 price * size 逐对相乘再累加
numerator = sum(df_book_TMF[p] * df_book_TMF[s] for p, s in zip(price_cols, size_cols))

# 2) 计算分母：所有 size 的和
denominator = df_book_TMF[size_cols].sum(axis=1)

# 3) 新增 bid_vwap
df_book_TMF['bid_vwap'] = numerator / denominator

In [57]:
cols_to_keep = [
    'bid_sz4', 'bid_sz3', 'bid_sz2', 'bid_sz1', 'bid_sz',
    'bid_px4', 'bid_px3', 'bid_px2', 'bid_px1', 'bid_px',
     'ask_px','bid_vwap'
]

In [ ]:
df_book_TXF = cmd.book_printer_v2('FUT_TAIFEX_TXF:202504', dt.date(2025,3,4),source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ',depth=1)



In [ ]:
df_book_TXF.between_time('21:30:03','21:30:06')[['ask_px','ask_sz']]

,ask_px,ask_sz
time,,
2025-03-07 21:30:03.004992,22439.0,1
2025-03-07 21:30:03.005348,22439.0,1
2025-03-07 21:30:03.005359,22439.0,1
2025-03-07 21:30:03.007745,22439.0,1
2025-03-07 21:30:03.011042,22439.0,1
2025-03-07 21:30:03.011042,22438.0,1
2025-03-07 21:30:03.011042,22439.0,1
2025-03-07 21:30:03.011042,22439.0,1
2025-03-07 21:30:03.011042,22439.0,1


In [64]:
df_tick_TXF = cmd.tick_printer('FUT_TAIFEX_TXF:202504',dt.date(2025,3,7))


In [68]:
df_tick_TXF.between_time('21:30:02','21:30:05')

,ex_time,instr,sz,px,bid_px,ask_px,bid_sz,ask_sz
time,,,,,,,,
2025-03-07 21:30:02.327508,2025-03-07 21:30:02.225,FUT_TAIFEX_TXF:202504,1,22376.0,22376.0,22444.0,1,4
2025-03-07 21:30:02.327508,2025-03-07 21:30:02.225,FUT_TAIFEX_TXF:202504,2,22375.0,22376.0,22444.0,1,4
2025-03-07 21:30:02.328025,2025-03-07 21:30:02.228,FUT_TAIFEX_TXF:202504,1,22371.0,22376.0,22444.0,1,4
2025-03-07 21:30:02.328045,2025-03-07 21:30:02.228,FUT_TAIFEX_TXF:202504,1,22371.0,22376.0,22444.0,1,4
2025-03-07 21:30:02.453418,2025-03-07 21:30:02.440,FUT_TAIFEX_TXF:202504,1,22380.0,22407.0,22454.0,2,1
2025-03-07 21:30:02.453685,2025-03-07 21:30:02.441,FUT_TAIFEX_TXF:202504,1,22420.0,22407.0,22454.0,2,1
2025-03-07 21:30:02.453825,2025-03-07 21:30:02.450,FUT_TAIFEX_TXF:202504,1,22379.0,22407.0,22454.0,2,1
2025-03-07 21:30:02.577327,2025-03-07 21:30:02.490,FUT_TAIFEX_TXF:202504,1,22370.0,22373.0,22440.0,1,6
2025-03-07 21:30:02.577363,2025-03-07 21:30:02.494,FUT_TAIFEX_TXF:202504,1,22369.0,22373.0,22440.0,1,6


In [ ]:
df_book_TMF.between_time('21:30:02','21:30:05')['bid_vwap']

time
2025-03-07 21:30:02.413255    22426.900000
2025-03-07 21:30:02.432059    22426.555556
2025-03-07 21:30:02.441086    22421.321429
2025-03-07 21:30:02.441710    22416.777778
2025-03-07 21:30:02.441710    22416.800000
2025-03-07 21:30:02.441710    22416.611111
2025-03-07 21:30:02.441710    22416.611111
2025-03-07 21:30:02.441710    22416.428571
2025-03-07 21:30:02.441710    22416.428571
2025-03-07 21:30:02.441710    22416.428571
2025-03-07 21:30:02.441710    22416.428571
2025-03-07 21:30:02.441710    22416.307692
2025-03-07 21:30:02.441710    22416.166667
2025-03-07 21:30:02.441710    22416.636364
2025-03-07 21:30:02.441710    22417.357143
2025-03-07 21:30:02.441710    22417.384615
2025-03-07 21:30:02.441710    22416.000000
2025-03-07 21:30:02.441710    22416.000000
2025-03-07 21:30:02.514403    22415.857143
2025-03-07 21:30:02.514403    22415.857143
2025-03-07 21:30:02.514403    22416.333333
2025-03-07 21:30:02.514403    22416.333333
2025-03-07 21:30:02.532679    22415.500000
2025-0

In [11]:
price_cols = ['bid_px4', 'bid_px3', 'bid_px2', 'bid_px1', 'bid_px']
size_cols  = ['bid_sz4', 'bid_sz3', 'bid_sz2', 'bid_sz1', 'bid_sz']

# 计算加权平均
# 1) 先用 price_cols 和 size_cols 一一对应相乘，再 sum 起来得到分子
num = df_book_TMF[price_cols].mul(df_book_TMF[size_cols]).sum(axis=1)
# 2) sum size_cols 得到分母
den = df_book_TMF[size_cols].sum(axis=1)
df_book_TMF['bid_vwap'] = num / den

In [ ]:
df_tick_TMF['px'] = pd.to_numeric(df_tick_TMF['px'], errors='coerce')
df_tick_TMF['bid_px'] = pd.to_numeric(df_tick_TMF['bid_px'], errors='coerce')
df_tick_TMF['ask_px'] = pd.to_numeric(df_tick_TMF['ask_px'], errors='coerce')

In [ ]:
import numpy as np

conditions = [
    df_tick_TMF['px'] <= df_tick_TMF['bid_px'],
    df_tick_TMF['px'] >= df_tick_TMF['ask_px']
]
choices = ['sell', 'buy']
df_tick_TMF['side'] = np.select(conditions, choices, default=None)


In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


In [ ]:
df_tick_TMF

In [ ]:

# 假設 txf_realfill 已經存在且 index 是時間
df_plot = df_book_TMF.copy()

# 確保 index 是 datetime（如果已經是就沒事）
df_plot.index = pd.to_datetime(df_plot.index)

# 把 bid_px 和 ask_px 中的 0 換成 NaN，這樣畫圖時會跳過
df_plot["bid_px"] = df_plot["bid_px"].replace(0, np.nan)
df_plot["ask_px"] = df_plot["ask_px"].replace(0, np.nan)

# 畫圖
plt.figure(figsize=(12, 6))
plt.plot(df_plot.index, df_plot["bid_px"], label="Bid Price", linewidth=1)
plt.plot(df_plot.index, df_plot["ask_px"], label="Ask Price", linewidth=1)

plt.xlabel("Time")
plt.ylabel("Price")
plt.title("Bid and Ask Prices over Time (0 Replaced with NaN)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()